In [1]:
import pandas as pd
from transformers import AutoTokenizer,DataCollatorWithPadding,Trainer,EarlyStoppingCallback
from transformers import AutoModelForSequenceClassification, TrainingArguments,SchedulerType
from transformers import trainer_utils
from transformers.trainer_utils import IntervalStrategy as SaveStrategy
from datasets import load_dataset
from datasets import Dataset, DatasetDict
import torch
from torch.serialization import add_safe_globals
import evaluate
import numpy as np
from sklearn.metrics import accuracy_score, log_loss, classification_report
from sklearn.utils.class_weight import compute_class_weight
from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss
from sklearn.model_selection import train_test_split
from typing import Dict, List, Tuple, Any
import matplotlib.pyplot as plt
import seaborn as sns
import re
import random
import math
import ipywidgets
import os



2025-08-11 03:38:17.138425: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754883497.161461    7122 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754883497.169135    7122 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1754883497.190350    7122 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1754883497.190383    7122 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1754883497.190389    7122 computation_placer.cc:177] computation placer alr

In [2]:
from datasets.utils.logging import enable_progress_bar
enable_progress_bar()

In [3]:
import sys, torch
print("Python:", sys.version)
print("PyTorch:", torch.__version__)
import transformers; print("Transformers:", transformers.__version__, transformers.__file__)
import subprocess; subprocess.run([sys.executable, "-m", "pip", "show", "transformers"])

Python: 3.10.12 (main, May 27 2025, 17:12:29) [GCC 11.4.0]
PyTorch: 2.7.0
Transformers: 4.55.0 /home/ubuntu/.local/lib/python3.10/site-packages/transformers/__init__.py
Name: transformers
Version: 4.55.0
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /home/ubuntu/.local/lib/python3.10/site-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: 


CompletedProcess(args=['/usr/bin/python3', '-m', 'pip', 'show', 'transformers'], returncode=0)

In [4]:
print("PyTorch version:", torch.__version__)
print("Transformers version:", transformers.__version__)

PyTorch version: 2.7.0
Transformers version: 4.55.0


In [5]:
import importlib
import finetuning_eval_func
importlib.reload(finetuning_eval_func)
from finetuning_eval_func import process_csv_social_bias


/usr/bin/python3
/usr/bin/python3


In [6]:
train_df, val_df, test_df, label2id, id2label = process_csv_social_bias('./')

Train size: 29790
Validation size: 3724
Test size: 3724


/home/ubuntu/finetuning_eval_func.py:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = df['bias_type'].map(label2id)


In [7]:
print("CUDA available:", torch.cuda.is_available())
print("GPU name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")
os.environ["TRANSFORMERS_NO_TF"] = "1"

CUDA available: True
GPU name: NVIDIA RTX A6000


In [8]:
print(torch.version.cuda)         # CUDA version PyTorch was built with
print(torch.backends.cudnn.version())  # cuDNN version
print(torch.cuda.is_available())

12.8
90800
True


In [9]:
def tokenize_function(examples):
    # use truncation only-collator handle padding dynamically
    return tokenizer(examples["post"], truncation=True)

### Loading the training arguments from the best cross entropy loss Hatebert model from sweep1_htebert.ipynb

In [10]:
def compute_class_weights(train_df, label2id):
    # correct type check if train_df['label'] are strings, map them to ids first
    y = train_df['label']
    if not np.issubdtype(y.dtype, np.number):
        y = y.map(label2id).values
    else:
        y = y.values

    classes = np.array(sorted(label2id.values()))
    weights = compute_class_weight(class_weight='balanced', classes=classes, y=y)
    return torch.tensor(weights, dtype=torch.float)

class WeightedTrainer(Trainer):
    def __init__(self, *args, class_weights=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.class_weights = class_weights  # torch tensor, move to device later

    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits  # [batch, num_labels] for sequence classification

        # Ensure weights are on the same device as logits
        if self.class_weights is not None:
            class_weights = self.class_weights.to(logits.device)
            loss_fct = CrossEntropyLoss(weight=class_weights)
        else:
            loss_fct = CrossEntropyLoss()

        # Flattening guards against token-classification shapes; harmless for seq cls
        loss = loss_fct(logits.view(-1, logits.size(-1)), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

In [24]:
accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy.compute(predictions=preds, references=labels)["accuracy"],
        "f1_macro": f1.compute(predictions=preds, references=labels, average="macro")["f1"],
    }

In [25]:
print("torch.version.cuda:", torch.version.cuda)       # None on CPU build
print("torch.cuda.is_available():", torch.cuda.is_available())  # False in CPU
print("CUDA_VISIBLE_DEVICES:", os.environ.get("CUDA_VISIBLE_DEVICES"))

torch.version.cuda: 12.8
torch.cuda.is_available(): True
CUDA_VISIBLE_DEVICES: None


NEW

In [26]:
# Device: CUDA if available, else CPU (explicitly avoid Metal)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

def set_seed(seed=42):
    random.seed(seed); np.random.seed(seed); torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)


Using device: cuda


In [36]:
#Loading from checkpoing and inheriting parameters
CKPT_DIR = "checkpoint-2236"  #  ".../checkpoint-2500"
DATA_DIR = "./"              # dir that contains social_bias.csv

torch.set_float32_matmul_precision("high")  # speeds up matmul on A6000
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load tokenizer & model 
tokenizer = AutoTokenizer.from_pretrained(CKPT_DIR, use_fast=True)
model = AutoModelForSequenceClassification.from_pretrained(CKPT_DIR).to(device)

# ---- STRICT hyperparam recovery (no defaults) ----
def load_training_args_best_effort(ckpt_dir):
    args_bin  = os.path.join(ckpt_dir, "training_args.bin")
    args_json = os.path.join(ckpt_dir, "training_args.json")
    hp = None

    if os.path.exists(args_bin):
        try:
            hp = torch.load(args_bin, map_location="cpu", weights_only=False)  # No safe_globals
            print("Loaded training_args from BIN.")
        except Exception as e:
            print("BIN load failed:", repr(e))

    if hp is None and os.path.exists(args_json):
        try:
            with open(args_json, "r") as f:
                hp = json.load(f)
            print("Loaded training_args from JSON.")
        except Exception as e:
            print("JSON load failed:", repr(e))

    # Normalize to dict; fill only what exists; tolerate missing keys
    def getv(obj, k):
        return obj.get(k, None) if isinstance(obj, dict) else getattr(obj, k, None)

    d = {}
    if hp is not None:
        d["learning_rate"]  = getv(hp, "learning_rate")
        d["weight_decay"]   = getv(hp, "weight_decay")
        # Some runs store warmup as steps, not ratio
        wr = getv(hp, "warmup_ratio")
        ws = getv(hp, "warmup_steps")
        d["warmup_ratio"]   = wr if wr is not None else (0.0 if ws else None)  # we’ll set our own anyway
        lst = getv(hp, "lr_scheduler_type")
        d["lr_scheduler_type"] = str(lst) if lst is not None else None
        # Nice-to-haves:
        for k in ["adam_beta1","adam_beta2","adam_epsilon","max_grad_norm",
                  "per_device_train_batch_size","per_device_eval_batch_size",
                  "gradient_accumulation_steps","eval_steps","logging_steps","save_steps"]:
            v = getv(hp, k)
            if v is not None:
                d[k] = v
    return {k:v for k,v in d.items() if v is not None}

saved_hp = load_training_args_best_effort(CKPT_DIR)
print("Recovered HPs (non-strict):", saved_hp)




Loaded training_args from BIN.
Recovered HPs (non-strict): {'learning_rate': 1.7011977627094936e-05, 'weight_decay': 0.01, 'warmup_ratio': 0.0, 'lr_scheduler_type': 'SchedulerType.LINEAR', 'adam_beta1': 0.9, 'adam_beta2': 0.999, 'adam_epsilon': 1e-08, 'max_grad_norm': 1.0, 'per_device_train_batch_size': 8, 'per_device_eval_batch_size': 8, 'gradient_accumulation_steps': 1, 'logging_steps': 500, 'save_steps': 500}


In [28]:
#text column your tokenize_function expects
TEXT_COL = "text" if "text" in train_df.columns else "post"

# label is clean ints-only if not done
for df_ in (train_df, val_df, test_df):
    assert "label" in df_.columns and df_["label"].notna().all(), "Labels must be present and non-NaN"
    df_["label"] = df_["label"].astype(int)

# build datasets (avoid index column)
train_dataset = Dataset.from_pandas(train_df[[TEXT_COL, "label"]], preserve_index=False)
val_dataset   = Dataset.from_pandas(val_df[[TEXT_COL, "label"]],   preserve_index=False)
test_dataset  = Dataset.from_pandas(test_df[[TEXT_COL, "label"]],  preserve_index=False)

# tokenize (tokenize_function  with batch[TEXT_COL])
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset   = val_dataset.map(tokenize_function, batched=True)
test_dataset  = test_dataset.map(tokenize_function, batched=True)

# keep only what Trainer/model need 
cols_to_keep = {"input_ids", "attention_mask", "label"}
train_dataset = train_dataset.remove_columns([c for c in train_dataset.column_names if c not in cols_to_keep])
val_dataset   = val_dataset.remove_columns([c for c in val_dataset.column_names   if c not in cols_to_keep])
test_dataset  = test_dataset.remove_columns([c for c in test_dataset.column_names if c not in cols_to_keep])

# correct label column name
for d in (train_dataset, val_dataset, test_dataset):
    if "label" in d.column_names and "labels" not in d.column_names:
        d = d.rename_column("label", "labels")


Map:   0%|          | 0/29790 [00:00<?, ? examples/s]

Map:   0%|          | 0/3724 [00:00<?, ? examples/s]

Map:   0%|          | 0/3724 [00:00<?, ? examples/s]

In [29]:
# precision choice (A6000 supports bf16 in latest drivers)

OUTPUT_DIR       = "./hatebert_resume_out_modified_try2"
NEW_LR           =  2e-5
NEW_EPOCHS       = 4
WARMUP_RATIO     = 0.10
SCHEDULER_TYPE   = SchedulerType.COSINE
CHECKS_PER_EPOCH = 6
PER_DEV_TRAIN_BS = 16
PER_DEV_EVAL_BS  = 32
GRAD_ACCUM       = 1

use_bf16 = torch.cuda.is_bf16_supported()

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, pad_to_multiple_of=8)

steps_per_epoch = math.ceil(len(train_dataset) / (PER_DEV_TRAIN_BS * max(1, GRAD_ACCUM)))
eval_save_steps = max(1, steps_per_epoch // CHECKS_PER_EPOCH)

args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=PER_DEV_TRAIN_BS,
    per_device_eval_batch_size=PER_DEV_EVAL_BS,
    gradient_accumulation_steps=GRAD_ACCUM,

    num_train_epochs=NEW_EPOCHS,
    learning_rate=NEW_LR,
    weight_decay=0.01,
    warmup_ratio=WARMUP_RATIO,
    lr_scheduler_type=SchedulerType.COSINE,   # ← use enum
    adam_beta1=0.9,
    adam_beta2=0.999,
    adam_epsilon=1e-8,
    max_grad_norm=1.0,

    eval_strategy="steps",
    logging_strategy="steps",
    eval_steps=eval_save_steps,
    save_steps=eval_save_steps,
    logging_steps=max(1, eval_save_steps // 2),

    save_total_limit=3,
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    greater_is_better=True,

    bf16=use_bf16,
    fp16=not use_bf16 and torch.cuda.is_available(),
    report_to="none",
)

trainer = Trainer(
    model=model,                          # Trainer handles device placement
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,          
    compute_metrics=compute_metrics,      # including "f1_macro"
    callbacks=[EarlyStoppingCallback(early_stopping_patience=6)],
)

/tmp/ipykernel_7122/3210279973.py:52: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [23]:
# Train (set resume=True for optimizer/scheduler state restored from CKPT_DIR)


resume = True
print("Will resume from:", CKPT_DIR)
trainer.train(resume_from_checkpoint=CKPT_DIR if resume else None)

# Validation
val_metrics = trainer.evaluate()
print("Validation:", val_metrics)

# Test -metrics only-
test_metrics = trainer.evaluate(eval_dataset=test_dataset)
print("Test:", test_metrics)


# See which checkpoint was best (since load_best_model_at_end=True)
print("Best checkpoint:", trainer.state.best_model_checkpoint)

# Save best model + tokenizer
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

Step,Training Loss,Validation Loss,Accuracy,F1 Macro
310,0.159700,1.129448,0.734962,0.516661
620,0.231000,1.150488,0.725295,0.495629
930,0.174700,1.338259,0.703545,0.498628
1240,0.172300,1.316735,0.712137,0.494760
1550,0.188100,1.370171,0.725832,0.491120
1860,0.160900,1.365913,0.717777,0.490827
2170,0.095100,1.671004,0.726369,0.487960


Validation: {'eval_loss': 1.1294480562210083, 'eval_accuracy': 0.7349624060150376, 'eval_f1_macro': 0.5166608887735815, 'eval_runtime': 1.886, 'eval_samples_per_second': 1974.509, 'eval_steps_per_second': 62.035, 'epoch': 1.1654135338345863}
Test: {'eval_loss': 1.1108424663543701, 'eval_accuracy': 0.7320085929108485, 'eval_f1_macro': 0.5108704789941567, 'eval_runtime': 2.1507, 'eval_samples_per_second': 1731.539, 'eval_steps_per_second': 54.401, 'epoch': 1.1654135338345863}
Best checkpoint: ./hatebert_resume_out_modified/checkpoint-310


('./hatebert_resume_out_modified/tokenizer_config.json',
 './hatebert_resume_out_modified/special_tokens_map.json',
 './hatebert_resume_out_modified/vocab.txt',
 './hatebert_resume_out_modified/added_tokens.json',
 './hatebert_resume_out_modified/tokenizer.json')